In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../")

In [3]:
from ease_recommender import *
from npmi_recommender import *

import pickle as p

In [4]:
print("loading cache data...")
D = p.load(open("cached_data/movie_lens_preprocessed.p", "rb"))

row = D["userId"]
col = D["movieId"]
data = D["rating"]

movies = D["movies"]

print("done")

loading cache data...
done


In [5]:
def find_match_using_terms(terms, movies=movies, case_insensitive=False):
    assert type(terms) in (list, tuple, set)
    
    title = movies.title
    if case_insensitive:
        title = title.str.lower()
    
    matches = True
    for term in terms:
        matches &= title.str.contains(term)
        
    matches = np.where(matches)[0]

    if len(matches) > 1:
        raise Exception("Multiple matches found, filter down to a single match", movies.loc[matches, "title"].tolist())
        
    return matches[0]

In [6]:
mat = csr_matrix((data.astype(bool), (row, col))).astype(np.int64)

In [147]:
# a = find_match_using_terms(["Sense and Sensibility", "1995"])
# a = find_match_using_terms(["Knives Out", "2019"])
# a = find_match_using_terms(["Witness", "1957"])
a = find_match_using_terms(["Immortal Beloved"])

movies.loc[a]

title         Immortal Beloved (1994)
genres                  Drama|Romance
imdbId                         110116
tmdbId                        13701.0
avg_rating                    7.33898
num_votes                        3250
Name: 246, dtype: object

In [148]:
# b = find_match_using_terms(["Pride and Prejudice", "1995"])
# b = find_match_using_terms(["Death", "Nile", "1978"])
b = find_match_using_terms(["Amadeus"])

movies.loc[b]

title         Amadeus (1984)
genres                 Drama
imdbId                 86879
tmdbId                 279.0
avg_rating          8.135853
num_votes              21756
Name: 1193, dtype: object

In [163]:
lambda_ = optimize_lambda_using_a_to_b_matching_npmi(mat, a, b, do_combine_npmi=True)

lambda_: 0.0
error: 36652.0
lambda_: 1.0
error: 16
lambda_: 2.6180339999999998
error: 17
lambda_: 1.6180339748439998
error: 18
lambda_: 0.618034
error: 10.618034
lambda_: 0.381966025156
error: 12.381966025156
lambda_: 0.6070858192270696
error: 11.60708581922707
lambda_: 0.763932025156
error: 14.763932025156
lambda_: 0.6737620850767367
error: 12.673762085076737
lambda_: 0.6393202337444208
error: 11.63932023374442
lambda_: 0.6230865037182408
error: 11.623086503718241
lambda_: 0.6150585036106236
error: 11.615058503610623
lambda_: 0.6199638846352415
error: 10.619963884635242
lambda_: 0.6189284466235736
error: 10.618928446623574
lambda_: 0.6168974615461355
error: 11.616897461546136
lambda_: 0.6175998809529312
error: 10.61759988095293
lambda_: 0.617331580621795
error: 10.617331580621794
lambda_: 0.6171657618949417
error: 10.617165761894942
lambda_: 0.6170632802839096
error: 11.61706328028391
lambda_: 0.617229099010763
error: 10.617229099010762
lambda_: 0.6171266174039022
error: 10.6171266174

In [165]:
lambda_

np.float64(0.6171118335416123)

In [166]:
# a_to_b_error_metric_npmi_pop_weighted(mat, a, b, lambda_, lambda_penalty=False)

In [167]:
# lambda_ = optimize_lambda_using_a_to_b_matching_npmi(mat, a, b, optimize_rank=False)

In [168]:
top_k = 20

In [169]:
cache_a=make_npmi_batch_popularity_weighted_cache(mat, a)
cache_b=make_npmi_batch_popularity_weighted_cache(mat, b)

cache_a, cache_b = combine_npmi(cache_a, cache_b, a, b)

In [170]:
# using normalized pointwise mutual information (popularity weighted)

similarity_scores = npmi_batch_popularity_weighted(mat, a, lambda_, cache=cache_a)
similarity_scores[a] = 0
similarity_scores[b] = 0

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

top_k_matches

,title,genres,imdbId,tmdbId,avg_rating,num_votes
movieId,,,,,,
296,Quiz Show (1994),Drama,110932,11450.0,7.321774,13676
352,Four Weddings and a Funeral (1994),Comedy|Romance,109831,712.0,7.284194,19987
504,"Piano, The (1993)",Drama|Romance,107822,713.0,7.368306,12289
492,Much Ado About Nothing (1993),Comedy|Romance,107616,11971.0,7.734885,11266
600,Fargo (1996),Comedy|Crime|Drama|Thriller,116282,275.0,8.230650,51507
49,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0,8.535730,64725
332,What's Eating Gilbert Grape (1993),Drama,108550,1587.0,7.491266,14175
582,Dances with Wolves (1990),Adventure|Drama|Western,99348,581.0,7.482605,34979
16,Sense and Sensibility (1995),Drama|Romance,114388,4584.0,7.891793,18677


In [144]:
for m in movies.loc[np.argsort(-similarity_scores)[:top_k], "title"]:
    print(m)

Anatomy of a Murder (1959)
Murder on the Orient Express (1974)
Sleuth (1972)
And Then There Were None (1945)
Evil Under the Sun (1982)
Man Who Knew Too Much, The (1956)
Lady Vanishes, The (1938)
To Have and Have Not (1944)
Kind Hearts and Coronets (1949)
Rope (1948)
Judgment at Nuremberg (1961)
Dial M for Murder (1954)
Sorry, Wrong Number (1948)
Sweet Smell of Success (1957)
Double Indemnity (1944)
Lavender Hill Mob, The (1951)
Suspicion (1941)
Murder by Death (1976)
Dark Passage (1947)
Postman Always Rings Twice, The (1946)
